# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [74]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [75]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [76]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [77]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [78]:
encoded[:100]

array([13, 57, 60, 77, 34, 43, 65, 54, 81, 22, 22, 22, 14, 60, 77, 77, 71,
       54, 49, 60, 39, 55, 20, 55, 43, 38, 54, 60, 65, 43, 54, 60, 20, 20,
       54, 60, 20, 55, 72, 43, 61, 54, 43, 21, 43, 65, 71, 54,  5, 45, 57,
       60, 77, 77, 71, 54, 49, 60, 39, 55, 20, 71, 54, 55, 38, 54,  5, 45,
       57, 60, 77, 77, 71, 54, 55, 45, 54, 55, 34, 38, 54, 32, 58, 45, 22,
       58, 60, 71, 35, 22, 22, 27, 21, 43, 65, 71, 34, 57, 55, 45])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [79]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [80]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [81]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: []\
    total_batch_size = batch_size * seq_length
    n_batches = (len(arr) - 1) // total_batch_size
    
    ## TODO: Keep only enough characters to make full batches
    size = n_batches * total_batch_size
    last = arr[size]
    arr = arr[:size]
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape(batch_size, -1)
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n: n + seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1] = arr[:, n+1: n + seq_length]
        try:
            y[:, -1] = arr[:, n + seq_length]
        except:
            y[:, -1] = np.append(arr[1:, 0], [last])
                
        yield x, y

In [82]:
test = np.array([1,2,3,4,5,6,7,8,9,10,11,12, 13, 14, 15, 16, 17, 18, 19])
print(test[:-1].reshape(3, -1))
for x, y in get_batches(test, 3, 2):
    print(x)
    print(y)
    

[[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]
 [13 14 15 16 17 18]]
[[ 1  2]
 [ 7  8]
 [13 14]]
[[ 2  3]
 [ 8  9]
 [14 15]]
[[ 3  4]
 [ 9 10]
 [15 16]]
[[ 4  5]
 [10 11]
 [16 17]]
[[ 5  6]
 [11 12]
 [17 18]]
[[ 6  7]
 [12 13]
 [18 19]]


### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [83]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [86]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[13 57 60 77 34 43 65 54 81 22]
 [38 32 45 54 34 57 60 34 54 60]
 [43 45 47 54 32 65 54 60 54 49]
 [38 54 34 57 43 54 52 57 55 43]
 [54 38 60 58 54 57 43 65 54 34]
 [52  5 38 38 55 32 45 54 60 45]
 [54  7 45 45 60 54 57 60 47 54]
 [69 12 20 32 45 38 72 71 35 54]]

y
 [[57 60 77 34 43 65 54 81 22 22]
 [32 45 54 34 57 60 34 54 60 34]
 [45 47 54 32 65 54 60 54 49 32]
 [54 34 57 43 54 52 57 55 43 49]
 [38 60 58 54 57 43 65 54 34 43]
 [ 5 38 38 55 32 45 54 60 45 47]
 [ 7 45 45 60 54 57 60 47 54 38]
 [12 20 32 45 38 72 71 35 54 75]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `dropout_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [87]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [124]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        num_characters = len(self.chars)
        
        ## TODO: define the layers of the model
        self.lstm1 = nn.LSTM(num_characters, n_hidden, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc1 = nn.Linear(n_hidden, num_characters)
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        x, hidden = self.lstm1(x, hidden)

        x = self.dropout(x)
        
        # x has dimentions [batch_size, sequence_length, n_hidden], we want it to have size [batch_size, seq_length]
        x = x.contiguous().view(-1, self.n_hidden)
        
        x = self.fc1(x)
        
        # return the final output and the hidden state
        return x, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [125]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [126]:
## TODO: set you model hyperparameters
# define and print the net
n_hidden=256
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm1): LSTM(83, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc1): Linear(in_features=256, out_features=83, bias=True)
)


### Set your training hyperparameters!

In [128]:
batch_size = 10
seq_length = 20
n_epochs = 5 # start small if you are just testing initial behavior

train_on_gpu = True

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/5... Step: 10... Loss: 3.0366... Val Loss: 3.1603
Epoch: 1/5... Step: 20... Loss: 3.2976... Val Loss: 3.1428
Epoch: 1/5... Step: 30... Loss: 3.1520... Val Loss: 3.1348
Epoch: 1/5... Step: 40... Loss: 3.1918... Val Loss: 3.1347
Epoch: 1/5... Step: 50... Loss: 3.0966... Val Loss: 3.1346
Epoch: 1/5... Step: 60... Loss: 3.1376... Val Loss: 3.1353
Epoch: 1/5... Step: 70... Loss: 3.2316... Val Loss: 3.1295
Epoch: 1/5... Step: 80... Loss: 3.1394... Val Loss: 3.1281
Epoch: 1/5... Step: 90... Loss: 3.1602... Val Loss: 3.1270
Epoch: 1/5... Step: 100... Loss: 3.1435... Val Loss: 3.1178
Epoch: 1/5... Step: 110... Loss: 3.1071... Val Loss: 3.1148
Epoch: 1/5... Step: 120... Loss: 3.2478... Val Loss: 3.1062
Epoch: 1/5... Step: 130... Loss: 2.9715... Val Loss: 3.0878
Epoch: 1/5... Step: 140... Loss: 3.0549... Val Loss: 3.0478
Epoch: 1/5... Step: 150... Loss: 3.0160... Val Loss: 3.0068
Epoch: 1/5... Step: 160... Loss: 3.0976... Val Loss: 2.9928
Epoch: 1/5... Step: 170... Loss: 2.7915... Val Lo

Epoch: 1/5... Step: 1380... Loss: 1.9085... Val Loss: 2.0884
Epoch: 1/5... Step: 1390... Loss: 2.0876... Val Loss: 2.0842
Epoch: 1/5... Step: 1400... Loss: 2.2335... Val Loss: 2.0791
Epoch: 1/5... Step: 1410... Loss: 2.1165... Val Loss: 2.0816
Epoch: 1/5... Step: 1420... Loss: 2.1328... Val Loss: 2.0761
Epoch: 1/5... Step: 1430... Loss: 1.9952... Val Loss: 2.0752
Epoch: 1/5... Step: 1440... Loss: 1.9808... Val Loss: 2.0681
Epoch: 1/5... Step: 1450... Loss: 1.9251... Val Loss: 2.0636
Epoch: 1/5... Step: 1460... Loss: 2.1036... Val Loss: 2.0648
Epoch: 1/5... Step: 1470... Loss: 2.0442... Val Loss: 2.0627
Epoch: 1/5... Step: 1480... Loss: 2.0749... Val Loss: 2.0630
Epoch: 1/5... Step: 1490... Loss: 2.0212... Val Loss: 2.0593
Epoch: 1/5... Step: 1500... Loss: 1.9679... Val Loss: 2.0654
Epoch: 1/5... Step: 1510... Loss: 2.2451... Val Loss: 2.0598
Epoch: 1/5... Step: 1520... Loss: 2.0771... Val Loss: 2.0559
Epoch: 1/5... Step: 1530... Loss: 2.0232... Val Loss: 2.0550
Epoch: 1/5... Step: 1540

Epoch: 1/5... Step: 2730... Loss: 1.8818... Val Loss: 1.8671
Epoch: 1/5... Step: 2740... Loss: 1.8079... Val Loss: 1.8632
Epoch: 1/5... Step: 2750... Loss: 2.0078... Val Loss: 1.8606
Epoch: 1/5... Step: 2760... Loss: 1.9888... Val Loss: 1.8562
Epoch: 1/5... Step: 2770... Loss: 2.0058... Val Loss: 1.8559
Epoch: 1/5... Step: 2780... Loss: 2.0325... Val Loss: 1.8645
Epoch: 1/5... Step: 2790... Loss: 1.7280... Val Loss: 1.8587
Epoch: 1/5... Step: 2800... Loss: 1.6429... Val Loss: 1.8537
Epoch: 1/5... Step: 2810... Loss: 1.8220... Val Loss: 1.8524
Epoch: 1/5... Step: 2820... Loss: 1.7806... Val Loss: 1.8506
Epoch: 1/5... Step: 2830... Loss: 1.9828... Val Loss: 1.8544
Epoch: 1/5... Step: 2840... Loss: 1.8385... Val Loss: 1.8514
Epoch: 1/5... Step: 2850... Loss: 1.8078... Val Loss: 1.8479
Epoch: 1/5... Step: 2860... Loss: 1.8350... Val Loss: 1.8456
Epoch: 1/5... Step: 2870... Loss: 2.1668... Val Loss: 1.8458
Epoch: 1/5... Step: 2880... Loss: 2.0681... Val Loss: 1.8448
Epoch: 1/5... Step: 2890

Epoch: 1/5... Step: 4080... Loss: 1.6503... Val Loss: 1.7722
Epoch: 1/5... Step: 4090... Loss: 1.7228... Val Loss: 1.7653
Epoch: 1/5... Step: 4100... Loss: 1.6668... Val Loss: 1.7590
Epoch: 1/5... Step: 4110... Loss: 1.6699... Val Loss: 1.7579
Epoch: 1/5... Step: 4120... Loss: 1.6700... Val Loss: 1.7605
Epoch: 1/5... Step: 4130... Loss: 1.8053... Val Loss: 1.7571
Epoch: 1/5... Step: 4140... Loss: 1.8278... Val Loss: 1.7591
Epoch: 1/5... Step: 4150... Loss: 1.5839... Val Loss: 1.7547
Epoch: 1/5... Step: 4160... Loss: 1.7780... Val Loss: 1.7550
Epoch: 1/5... Step: 4170... Loss: 1.7443... Val Loss: 1.7604
Epoch: 1/5... Step: 4180... Loss: 1.8138... Val Loss: 1.7584
Epoch: 1/5... Step: 4190... Loss: 1.9604... Val Loss: 1.7606
Epoch: 1/5... Step: 4200... Loss: 1.8232... Val Loss: 1.7579
Epoch: 1/5... Step: 4210... Loss: 1.7338... Val Loss: 1.7582
Epoch: 1/5... Step: 4220... Loss: 1.8518... Val Loss: 1.7550
Epoch: 1/5... Step: 4230... Loss: 1.6920... Val Loss: 1.7659
Epoch: 1/5... Step: 4240

Epoch: 1/5... Step: 5430... Loss: 1.5923... Val Loss: 1.6886
Epoch: 1/5... Step: 5440... Loss: 1.8444... Val Loss: 1.6891
Epoch: 1/5... Step: 5450... Loss: 1.6403... Val Loss: 1.6948
Epoch: 1/5... Step: 5460... Loss: 1.6959... Val Loss: 1.6954
Epoch: 1/5... Step: 5470... Loss: 1.7297... Val Loss: 1.6913
Epoch: 1/5... Step: 5480... Loss: 1.6985... Val Loss: 1.6956
Epoch: 1/5... Step: 5490... Loss: 1.7845... Val Loss: 1.6955
Epoch: 1/5... Step: 5500... Loss: 1.9238... Val Loss: 1.6963
Epoch: 1/5... Step: 5510... Loss: 1.5704... Val Loss: 1.6906
Epoch: 1/5... Step: 5520... Loss: 1.7807... Val Loss: 1.6878
Epoch: 1/5... Step: 5530... Loss: 1.7294... Val Loss: 1.6887
Epoch: 1/5... Step: 5540... Loss: 1.7941... Val Loss: 1.6866
Epoch: 1/5... Step: 5550... Loss: 1.6649... Val Loss: 1.6883
Epoch: 1/5... Step: 5560... Loss: 1.9146... Val Loss: 1.6906
Epoch: 1/5... Step: 5570... Loss: 1.7403... Val Loss: 1.6884
Epoch: 1/5... Step: 5580... Loss: 1.7820... Val Loss: 1.6849
Epoch: 1/5... Step: 5590

...

Epoch: 5/5... Step: 38740... Loss: 1.5008... Val Loss: 1.4125
Epoch: 5/5... Step: 38750... Loss: 1.2086... Val Loss: 1.4125
Epoch: 5/5... Step: 38760... Loss: 1.4865... Val Loss: 1.4120
Epoch: 5/5... Step: 38770... Loss: 1.3633... Val Loss: 1.4135
Epoch: 5/5... Step: 38780... Loss: 1.4584... Val Loss: 1.4125
Epoch: 5/5... Step: 38790... Loss: 1.3485... Val Loss: 1.4110
Epoch: 5/5... Step: 38800... Loss: 1.2378... Val Loss: 1.4099
Epoch: 5/5... Step: 38810... Loss: 1.3275... Val Loss: 1.4123
Epoch: 5/5... Step: 38820... Loss: 1.3154... Val Loss: 1.4113
Epoch: 5/5... Step: 38830... Loss: 1.5002... Val Loss: 1.4102
Epoch: 5/5... Step: 38840... Loss: 1.4071... Val Loss: 1.4100
Epoch: 5/5... Step: 38850... Loss: 1.2642... Val Loss: 1.4107
Epoch: 5/5... Step: 38860... Loss: 1.4931... Val Loss: 1.4113
Epoch: 5/5... Step: 38870... Loss: 1.1811... Val Loss: 1.4100
Epoch: 5/5... Step: 38880... Loss: 1.3198... Val Loss: 1.4096
Epoch: 5/5... Step: 38890... Loss: 1.2753... Val Loss: 1.4091
Epoch: 5

Epoch: 5/5... Step: 40070... Loss: 1.2920... Val Loss: 1.4071
Epoch: 5/5... Step: 40080... Loss: 1.3837... Val Loss: 1.4096
Epoch: 5/5... Step: 40090... Loss: 1.5708... Val Loss: 1.4125
Epoch: 5/5... Step: 40100... Loss: 1.5982... Val Loss: 1.4148
Epoch: 5/5... Step: 40110... Loss: 1.2163... Val Loss: 1.4134
Epoch: 5/5... Step: 40120... Loss: 1.3274... Val Loss: 1.4116
Epoch: 5/5... Step: 40130... Loss: 1.2476... Val Loss: 1.4115
Epoch: 5/5... Step: 40140... Loss: 1.1374... Val Loss: 1.4105
Epoch: 5/5... Step: 40150... Loss: 1.3954... Val Loss: 1.4112
Epoch: 5/5... Step: 40160... Loss: 1.1100... Val Loss: 1.4103
Epoch: 5/5... Step: 40170... Loss: 1.0585... Val Loss: 1.4083
Epoch: 5/5... Step: 40180... Loss: 1.3205... Val Loss: 1.4094
Epoch: 5/5... Step: 40190... Loss: 1.2312... Val Loss: 1.4111
Epoch: 5/5... Step: 40200... Loss: 1.6220... Val Loss: 1.4084
Epoch: 5/5... Step: 40210... Loss: 1.5727... Val Loss: 1.4067
Epoch: 5/5... Step: 40220... Loss: 1.3445... Val Loss: 1.4054
Epoch: 5

Epoch: 5/5... Step: 41400... Loss: 1.3430... Val Loss: 1.4081
Epoch: 5/5... Step: 41410... Loss: 1.3342... Val Loss: 1.4077
Epoch: 5/5... Step: 41420... Loss: 1.2046... Val Loss: 1.4082
Epoch: 5/5... Step: 41430... Loss: 1.3860... Val Loss: 1.4096
Epoch: 5/5... Step: 41440... Loss: 1.2616... Val Loss: 1.4121
Epoch: 5/5... Step: 41450... Loss: 1.3678... Val Loss: 1.4118
Epoch: 5/5... Step: 41460... Loss: 1.5089... Val Loss: 1.4132
Epoch: 5/5... Step: 41470... Loss: 1.3224... Val Loss: 1.4151
Epoch: 5/5... Step: 41480... Loss: 1.3877... Val Loss: 1.4143
Epoch: 5/5... Step: 41490... Loss: 1.2650... Val Loss: 1.4132
Epoch: 5/5... Step: 41500... Loss: 1.3783... Val Loss: 1.4105
Epoch: 5/5... Step: 41510... Loss: 1.5263... Val Loss: 1.4082
Epoch: 5/5... Step: 41520... Loss: 1.3179... Val Loss: 1.4070
Epoch: 5/5... Step: 41530... Loss: 1.6065... Val Loss: 1.4072
Epoch: 5/5... Step: 41540... Loss: 1.4652... Val Loss: 1.4073
Epoch: 5/5... Step: 41550... Loss: 1.3476... Val Loss: 1.4093
Epoch: 5

Epoch: 5/5... Step: 42730... Loss: 1.3535... Val Loss: 1.4128
Epoch: 5/5... Step: 42740... Loss: 1.4086... Val Loss: 1.4119
Epoch: 5/5... Step: 42750... Loss: 1.3144... Val Loss: 1.4084
Epoch: 5/5... Step: 42760... Loss: 1.3340... Val Loss: 1.4065
Epoch: 5/5... Step: 42770... Loss: 1.2480... Val Loss: 1.4054
Epoch: 5/5... Step: 42780... Loss: 1.4089... Val Loss: 1.4075
Epoch: 5/5... Step: 42790... Loss: 1.3604... Val Loss: 1.4075
Epoch: 5/5... Step: 42800... Loss: 1.3925... Val Loss: 1.4079
Epoch: 5/5... Step: 42810... Loss: 1.4028... Val Loss: 1.4073
Epoch: 5/5... Step: 42820... Loss: 1.4642... Val Loss: 1.4075
Epoch: 5/5... Step: 42830... Loss: 1.6145... Val Loss: 1.4086
Epoch: 5/5... Step: 42840... Loss: 1.3416... Val Loss: 1.4092
Epoch: 5/5... Step: 42850... Loss: 1.3232... Val Loss: 1.4110
Epoch: 5/5... Step: 42860... Loss: 1.2613... Val Loss: 1.4128
Epoch: 5/5... Step: 42870... Loss: 1.2135... Val Loss: 1.4125
Epoch: 5/5... Step: 42880... Loss: 0.9885... Val Loss: 1.4124
Epoch: 5

Epoch: 5/5... Step: 44060... Loss: 1.2094... Val Loss: 1.4063
Epoch: 5/5... Step: 44070... Loss: 1.3790... Val Loss: 1.4060
Epoch: 5/5... Step: 44080... Loss: 1.3892... Val Loss: 1.4062
Epoch: 5/5... Step: 44090... Loss: 1.0124... Val Loss: 1.4080
Epoch: 5/5... Step: 44100... Loss: 1.2323... Val Loss: 1.4121
Epoch: 5/5... Step: 44110... Loss: 1.3599... Val Loss: 1.4150
Epoch: 5/5... Step: 44120... Loss: 1.1520... Val Loss: 1.4145
Epoch: 5/5... Step: 44130... Loss: 1.5067... Val Loss: 1.4109
Epoch: 5/5... Step: 44140... Loss: 1.1655... Val Loss: 1.4081
Epoch: 5/5... Step: 44150... Loss: 1.4484... Val Loss: 1.4072
Epoch: 5/5... Step: 44160... Loss: 1.3374... Val Loss: 1.4077
Epoch: 5/5... Step: 44170... Loss: 1.2422... Val Loss: 1.4075
Epoch: 5/5... Step: 44180... Loss: 1.2296... Val Loss: 1.4064
Epoch: 5/5... Step: 44190... Loss: 1.3537... Val Loss: 1.4051
Epoch: 5/5... Step: 44200... Loss: 1.5530... Val Loss: 1.4051
Epoch: 5/5... Step: 44210... Loss: 1.3955... Val Loss: 1.4059
Epoch: 5

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [129]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [130]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [131]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [132]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna stopped at his face.

"Well, that's not thrust
all the well, but I shall come by himself, where is seredy that there are a man stay in the mercal and the same same
thing to be so taking to all," said Levin affected to him of the point of him. Stepan Arkadyevitch, sters one was that he was so
much to be beant to him about that seems of the motter
were an ear of the position what an
tone to all how happy the sention to
hope.

"You'll take the friends, what a same, that is that in you, a disartroums. We'll be service to
allow that tog that seeing at
their
first well. Have the same tone?" and anyone
was thinking.
"He's not some of hers in the sort of at the side of take an also to be starking and all the second anything, with him, and I have too, when here so it are.

"I cannot take his footman as as I have an only think about that."

Then so he
saw she had said that always when which had so bought on another same soul at her husband, she
could not have shall
to speak of and were sank

## Loading a checkpoint

In [ ]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [ ]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))